## Importando Dataset - Campeonato Brasileiro

In [54]:
import pandas as pd

csv_path = '.\dataset\campeonato-brasileiro-full.csv'

df = pd.read_csv(csv_path, delimiter=';')

## Capturando informações gerais das colunas

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8440 entries, 0 to 8439
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                8440 non-null   int64  
 1   Rodada            8440 non-null   object 
 2   Data              8436 non-null   object 
 3   Horário           8079 non-null   object 
 4   Dia               8436 non-null   object 
 5   Mandante          8440 non-null   object 
 6   Visitante         8440 non-null   object 
 7   Vencedor          8440 non-null   object 
 8   Arena             8436 non-null   object 
 9   Mandante Placar   8436 non-null   float64
 10  Visitante Placar  8436 non-null   float64
 11  Estado Mandante   8440 non-null   object 
 12  Estado Visitante  8440 non-null   object 
 13  Estado Vencedor   8440 non-null   object 
dtypes: float64(2), int64(1), object(11)
memory usage: 923.2+ KB


## Listando os 5 primeiros registros

In [56]:
df.head(5)

,ID,Rodada,Data,Horário,Dia,Mandante,Visitante,Vencedor,Arena,Mandante Placar,Visitante Placar,Estado Mandante,Estado Visitante,Estado Vencedor
0,1,1,2000-07-29,16h00,Sábado,Fluminense,Bahia,Fluminense,Maracanã,2.0,0.0,RJ,BA,RJ
1,2,1,2000-07-29,16h00,Sábado,Vasco,Sport,Sport,São Januário,0.0,2.0,RJ,PE,PE
2,3,1,2000-07-29,16h00,Sábado,Vitória,Palmeiras,Vitória,Barradão,4.0,1.0,ES,SP,ES
3,4,1,2000-07-30,17h00,Domingo,Botafogo-RJ,Atlético-MG,-,Caio Martins,0.0,0.0,RJ,MG,-
4,5,1,2000-07-30,18h30,Domingo,Juventude,Flamengo,-,Alfredo Jaconi,1.0,1.0,RS,RJ,-


## Tratando dados inválidos

In [57]:
from unicodedata import normalize

def trataString(value):
    return normalize('NFKD', str(value)).encode('ASCII', 'ignore').decode('ASCII').upper()

df['Mandante'] = df['Mandante'].apply(trataString)
df['Visitante'] = df['Visitante'].apply(trataString)
df['Dia'] = df['Dia'].apply(trataString)
df['Vencedor'] = df['Vencedor'].apply(trataString)
df['Arena'] = df['Arena'].apply(trataString)

## Implemente pelo menos um filtro

## Ordenando o Dataset pelo maiores pontuadores independente de temporadas

In [58]:
df['Pontos Mandante'] = 0
df['Pontos Visitante'] = 0

df.loc[ df['Vencedor'] == df['Mandante'], 'Pontos Mandante' ] = 3
df.loc[ df['Vencedor'] == df['Visitante'], 'Pontos Visitante' ] = 3
df.loc[ df['Vencedor'] == '-', 'Pontos Mandante' ] = 1
df.loc[ df['Vencedor'] == '-', 'Pontos Visitante' ] = 1

times = pd.Series(df['Mandante'].unique(), name='Times')
tg = pd.DataFrame(times)
tg['Pontos'] = 0

mandante = df.groupby('Mandante')['Pontos Mandante'].sum()
visitante = df.groupby('Visitante')['Pontos Visitante'].sum()
nova = pd.DataFrame(mandante)
nova_2 = pd.DataFrame(visitante)

for index, i in nova.iterrows():
    tg.loc[ tg['Times'] == index, 'Pontos' ] += i['Pontos Mandante']

for index, i in nova_2.iterrows():
    tg.loc[ tg['Times'] == index, 'Pontos' ] += i['Pontos Visitante']

tg = tg.sort_values(by="Pontos", ascending=False).reset_index()
tg[['Times', 'Pontos']]

,Times,Pontos
0,SAO PAULO,1318
1,SANTOS,1236
2,FLAMENGO,1192
3,INTERNACIONAL,1172
4,GREMIO,1169
5,FLUMINENSE,1156
6,CRUZEIRO,1152
7,CORINTHIANS,1151
8,ATLETICO-MG,1117
9,ATHLETICO-PR,1115


## Ordenando o Dataset pelas maiores goleadas

In [59]:
df['Diferenca Gols'] = abs(df['Mandante Placar'] - df['Visitante Placar'])

# dropando valores nulos, visto que se não houve o jogo, provavelmente foi remarcado e está em outro registro
df = df.dropna(subset=['Diferenca Gols'])

df[['Mandante', 'Visitante', 'Vencedor', 'Diferenca Gols']].sort_values(by=['Diferenca Gols'], ascending=False)

,Mandante,Visitante,Vencedor,Diferenca Gols
1600,BAHIA,CRUZEIRO,CRUZEIRO,7.0
1122,GOIAS,JUVENTUDE,GOIAS,7.0
1992,SAO PAULO,PAYSANDU,SAO PAULO,7.0
1528,FIGUEIRENSE,PAYSANDU,FIGUEIRENSE,6.0
1338,FLAMENGO,BAHIA,FLAMENGO,6.0
...,...,...,...,...
2740,FLUMINENSE,SAO CAETANO,-,0.0
3750,SANTOS,NAUTICO,-,0.0
1666,JUVENTUDE,ATLETICO-MG,-,0.0
6932,VASCO,SANTOS,-,0.0


## Exibir as 5 maiores contagens de valores únicos de uma única coluna
#### No caso: Quem são os maiores vencedores em casa

In [60]:
df[ df['Mandante'] == df['Vencedor'] ][['Mandante']].value_counts().head()

Mandante     
SANTOS           233
SAO PAULO        231
GREMIO           223
ATHLETICO-PR     221
INTERNACIONAL    217
dtype: int64

## Implemente pelo menos um filtro

## Utilize, no mínimo, uma função agregada com agrupamento

## Exiba, no mínimo, três plots. Se possível, use um para variáveis quantitativa e outro para variáveis de qualitativas

In [63]:
df.groupby(by=['Estado Mandante'])['Mandante Placar'].sum() + df.groupby(by=['Estado Visitante'])['Visitante Placar'].sum()

Estado Mandante
AL      24.0
BA     521.0
CE     477.0
DF     141.0
ES     615.0
GO    1061.0
MG    2438.0
MT      12.0
PA     229.0
PE     880.0
PR    2147.0
RJ    3969.0
RN      24.0
RS    2426.0
SC    1269.0
SP    6011.0
dtype: float64

In [64]:
import datetime
import plotly.express as px

df['Data2'] = pd.to_datetime(df['Data'])
df['Ano'] = df['Data2'].dt.year
df.drop(columns='Data2', inplace=True)

times = pd.Series(df['Mandante'].unique(), name='Times')
ano = pd.Series(df['Ano'].unique(), name='Ano')
tg = pd.DataFrame(columns=['Time', 'Ano'])

count = 0
for i in times.to_frame().iterrows():
    for y in ano.to_frame().iterrows():
        count += 1
        tg.loc[-1, 'Time'] = i[1][0]
        tg.loc[-1, 'Ano'] = y[1][0]
        tg.index += count


tg['Pontos'] = 0
tg = tg.reset_index()

mandante = df.groupby(['Mandante', 'Ano'])['Pontos Mandante'].sum()
visitante = df.groupby(['Visitante', 'Ano'])['Pontos Visitante'].sum()
nova = pd.DataFrame(mandante)
nova_2 = pd.DataFrame(visitante)

for index, i in nova.iterrows():
    op_1 = tg['Time'] == index[0]
    op_2 = tg['Ano'] == index[1]
    tg.loc[op_1 & op_2, 'Pontos' ] += i['Pontos Mandante']

for index, i in nova_2.iterrows():
    op_1 = tg['Time'] == index[0]
    op_2 = tg['Ano'] == index[1]
    tg.loc[op_1 & op_2, 'Pontos' ] += i['Pontos Visitante']

grafic_3 = tg.groupby('Time')[['Time', 'Pontos']].mean().sort_values('Pontos', ascending=False).head(10)

time_filtro = 'FLAMENGO'
ano_filtro = tg['Ano'] == 2021
pontos_filtro = tg['Pontos'] != 0

# Grafico do time nas temporadas
grafic_1 = tg[ tg['Time'] == time_filtro ]

# grafico da tabela do temporada
grafic_2 = tg[ ano_filtro & pontos_filtro ]

grafic_1 = grafic_1.sort_values(by="Ano").reset_index()
grafic_2 = grafic_2.sort_values(by="Pontos")

grafico = px.line(grafic_1, x='Ano', y='Pontos')
grafico.show()

grafico = px.bar(grafic_2, x='Time', y='Pontos')
grafico.show()

grafico = px.bar(grafic_3, x=grafic_3.index, y='Pontos')
grafico.show()


